<a href="https://colab.research.google.com/github/trungduyen0220/text-classification-assignment/blob/master/Assignment1_NguyenThiMyDuyen_MSE13028.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading & preprocessing data

## 1. Downloading the dataset

In [1]:
!rm -f train.csv test.csv
!wget https://raw.githubusercontent.com/minhpqn/NLP-Notes/master/data/arxiv/train.csv
!wget https://raw.githubusercontent.com/minhpqn/NLP-Notes/master/data/arxiv/test.csv

--2020-09-06 18:14:58--  https://raw.githubusercontent.com/minhpqn/NLP-Notes/master/data/arxiv/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10400749 (9.9M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]   9.92M  19.7MB/s    in 0.5s    

2020-09-06 18:14:58 (19.7 MB/s) - ‘train.csv’ saved [10400749/10400749]

--2020-09-06 18:14:58--  https://raw.githubusercontent.com/minhpqn/NLP-Notes/master/data/arxiv/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1133794 (1.1M) [text/plain]
Saving

## 2. Importing library

In [2]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')

import pandas as pd
import numpy as np
import csv
import zipfile
import glob
from string import digits
import random

import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus.reader.wordnet import NOUN, VERB, ADJ
from sklearn.feature_extraction.text import CountVectorizer
english_stops = set(stopwords.words('english'))

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
# from keras.models import Model
# from keras.layers import Dense, Input, Dropout, LSTM, Activation
# from keras.layers.embeddings import Embedding
# from keras.preprocessing import sequence
# from keras.initializers import glorot_uniform

from collections import defaultdict
import math

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Let's see some rows in the data frame `train_df`.

In [3]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_df.head()

,id,title,abstract,category
0,1901.05003,Quantum Teleportation-Inspired Algorithm for S...,We show that low-depth random quantum circui...,quant-ph
1,1903.08905,RAP-Net: Recurrent Attention Pooling Networks ...,The response selection has been an emerging ...,cs
2,1907.11533,Combinatorial protein-protein interactions on ...,Scaffold proteins organize cellular processe...,q-bio
3,1711.07582,CVXR: An R Package for Disciplined Convex Opti...,CVXR is an R package that provides an object...,stat
4,1910.05833,Analytical Solution of (2+1) Dimensional Dirac...,"In this article, we studied the system of (2...",quant-ph


## 3. Cleaning and processing text data

In [4]:
def preformat(sentence):
  
  # Clean text data
  sentence = sentence.lower() # Lower the text sentence
  sentence = sentence.translate(str.maketrans('', '', digits)) # Delete digits number in text sentence
  sentence = re.sub(r"http\S+", "", sentence) # Remove URL in text sentence
  sentence = sentence.translate(str.maketrans('', '', string.punctuation)) # Remove punctuations in text sentence
  sentence = sentence.translate ({ord(c): " " for c in "!@#$%^&*()[]{}/<>?\\|`~=_+'"""})  # Remove special character in text sentence

  # Processing text data with NLP
  tokens = word_tokenize(sentence)
  tags = nltk.pos_tag(tokens)

  lemmatizer = WordNetLemmatizer()
  words = ""
  for i, token in enumerate(tokens):
    pos_tag = tags[i][1]
    if pos_tag.startswith('N'):
        lemma = lemmatizer.lemmatize(token, pos=NOUN)
        words += lemma + " "
    elif pos_tag.startswith('V'):
        lemma = lemmatizer.lemmatize(token, pos=VERB)
        words += lemma + " "
    elif pos_tag.startswith('J'):
        lemma = lemmatizer.lemmatize(token, pos=ADJ)
        words += lemma + " "
    else:
        lemma = token
        words += " "
  
  return ' '.join(words.split())

In [5]:
# Text before formatting
print("---Text before formatting: ")
print(train_df['abstract'][0])
# Text after formatting
print("---Text after formatting: ")
print(preformat(train_df['abstract'][0]))

---Text before formatting: 
  We show that low-depth random quantum circuits can be efficiently simulated by a quantum teleportation-inspired algorithm. By using logical qubits to redirect and teleport the quantum information in quantum circuits, the original circuits can be renormalized to new circuits with a smaller number of logical qubits. We demonstrate the algorithm to simulate several random quantum circuits, including 1D-chain 1000-qubit 42-depth, 2D-grid 125*8-qubit 42-depth and 2D-Bristlecone 72-qubit 32-depth circuits. Our results present a memory-efficient method with a clear physical picture to simulate low-depth random quantum circuits. 
---Text after formatting: 
show lowdepth random quantum circuit be simulate quantum teleportationinspired algorithm use logical qubits redirect teleport quantum information quantum circuit original circuit be renormalize new circuit small number logical qubits demonstrate algorithm simulate several random quantum circuit include dchain qu

## 4. Save the data into X_train, Y_train, X_test, Y_test

In [6]:
def read_csv(filename):
    abstract = []
    category = []

    with open (filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile )

        for row in csvReader:
            # Using preformat function to processing text data before training
            abstract.append(preformat(row[2]))
            category.append(row[3])
    
    # Remove the title
    abstract.pop(0)
    category.pop(0)
    # Transform to array
    X = np.asarray(abstract)
    Y = np.asarray(category)

    return X, Y

X_train, Y_train = read_csv('train.csv')
X_test, Y_test = read_csv('test.csv')

It takes a litle time to train because of lemmatizer processing.

# Training model

In this section, I will implement the Multinomial Naive Bayes (MNB) model.

Firstly, extract a vocabulary from a training dataset which is a list of sentences.

## 1. Build vocabulary function

In [7]:
def build_vocab(X):
    """
    Build vocabulary from dataset
    """
    vocab = {}
    for sentences in X:
        for word in sentences.split():
            # Check if word is a punctuation
            
            if word in string.punctuation:
                continue
            if word not in vocab:
                idx = len(vocab)
                vocab[word] = idx
    return vocab

Let's check how the function `build_vocab` works.

In [8]:
list_vocab = build_vocab(X_train);
# Select randomly 20 vocab in X_train and the number of occurrences of it in X_train
for i, val in enumerate(random.sample(list(list_vocab), 20)):
    print(val + ": " + str(list_vocab[val]))
  

selfinterference: 8311
hath: 35842
degeneracybreaking: 26555
singleplanet: 22596
koreas: 39931
pieta: 17447
tiled: 35930
shadowing: 17254
flexural: 13652
brbrightarrow: 39751
beampolarization: 11121
grouped: 7233
pathogen: 11136
tile: 8478
radmcd: 28402
ffcnn: 25442
sfss: 25043
myerson: 18344
metaparticle: 36729
energy: 758


## 2. Building the Multinomial Naive Bayes model


In [9]:
def train_naive_bayes(texts, labels, target_classes, alpha=1):
    """
    Train a multinomial Naive Bayes model
    """
    ndoc = 0
    nc = defaultdict(int)   # map from a class label to number of documents in the class
    logprior = dict()
    loglikelihood = dict()
    count = defaultdict(int)  # count the occurrences of w in documents of class c

    vocab = build_vocab(texts)
    # Training
    for s, c in zip(texts, labels):
        ndoc += 1
        nc[c] += 1
        for w in s.split():
            if w in vocab:
                count[(w,c)] += 1
    
    vocab_size = len(vocab)
    for c in target_classes:
        logprior[c] = math.log(nc[c]/ndoc)
        sum_ = 0
        for w in vocab.keys():
            if (w,c) not in count: count[(w,c)] = 0
            sum_ += count[(w,c)]
        
        for w in vocab.keys():
            loglikelihood[(w,c)] = math.log( (count[(w,c)] + alpha) / (sum_ + alpha * vocab_size) )
    
    return logprior, loglikelihood, vocab

Set the `target_classes` by adding ten unique categories of dataset.

In [10]:
target_classes = set()
for index in Y_train:
    target_classes.add(index)

print(target_classes)

{'eess', 'hep-th', 'q-bio', 'cs', 'math', 'stat', 'cond-mat', 'hep-ph', 'quant-ph', 'astro-ph'}


### Time to train

In [11]:
logprior, loglikelihood, vocab = train_naive_bayes(X_train, Y_train, target_classes)

## 3. Predict function

In [12]:
def test_naive_bayes(testdoc, logprior, loglikelihood, target_classes, vocab):
    sum_ = {}
    for c in  target_classes:
        sum_[c] = logprior[c]
        for w in testdoc.split():
            if w in vocab:
                sum_[c] += loglikelihood[(w,c)]
    # sort keys in sum_ by value
    sorted_keys = sorted(sum_.keys(), key=lambda x: sum_[x], reverse=True)
    return sorted_keys[0]

# Evaluation

## 1. Accuracy score

In [13]:
predicted_labels = [test_naive_bayes(s, logprior, loglikelihood, target_classes, vocab)
                    for s in X_test]

print('Accuracy score: %f' % metrics.accuracy_score(Y_test, predicted_labels))

Accuracy score: 0.892000


## 2. Calculate precision, recall, f1_score

In [14]:
print('  Precision: %f' % metrics.precision_score(Y_test, predicted_labels, average='macro'))
print('  Recall: %f' % metrics.recall_score(Y_test, predicted_labels, average='macro'))
print('  F1: %f' % metrics.f1_score(Y_test, predicted_labels, average='macro'))

  Precision: 0.893277
  Recall: 0.892000
  F1: 0.892093


## 3. Compute macro-averaged and micro-averaged f1 score

In [15]:
print('Macro-averaged f1: %f' % metrics.f1_score(Y_test, predicted_labels, average='macro'))
print('Micro-averaged f1: %f' % metrics.f1_score(Y_test, predicted_labels, average='micro'))

Macro-averaged f1: 0.892093
Micro-averaged f1: 0.892000


## 4. Report classification results all by once.

In [16]:
print(metrics.classification_report(Y_test, predicted_labels, digits=3))

              precision    recall  f1-score   support

    astro-ph      0.990     0.970     0.980       100
    cond-mat      0.902     0.920     0.911       100
          cs      0.738     0.790     0.763       100
        eess      0.839     0.780     0.808       100
      hep-ph      0.958     0.920     0.939       100
      hep-th      0.892     0.910     0.901       100
        math      0.902     0.830     0.865       100
       q-bio      0.895     0.940     0.917       100
    quant-ph      0.929     0.920     0.925       100
        stat      0.887     0.940     0.913       100

    accuracy                          0.892      1000
   macro avg      0.893     0.892     0.892      1000
weighted avg      0.893     0.892     0.892      1000

